In [2]:
import pandas as pd


In [5]:
df = pd.read_json("airyards.json")
df.head(10)

,name,position,team,week1,week10,week11,week12,week13,week14,week15,week16,week17,week2,week3,week4,week5,week6,week7,week8,week9
0,A.J. Brown,WR,TEN,"{'targets': 8, 'rec': 5, 'rec_yards': 39, 'air...","{'targets': 4, 'rec': 1, 'rec_yards': 21, 'air...","{'targets': 7, 'rec': 4, 'rec_yards': 62, 'air...","{'targets': 6, 'rec': 4, 'rec_yards': 98, 'air...","{'targets': 7, 'rec': 4, 'rec_yards': 87, 'air...","{'targets': 9, 'rec': 7, 'rec_yards': 112, 'ai...","{'targets': 6, 'rec': 5, 'rec_yards': 44, 'air...","{'targets': 8, 'rec': 4, 'rec_yards': 43, 'air...","{'targets': 11, 'rec': 10, 'rec_yards': 151, '...",NaN,NaN,NaN,"{'targets': 9, 'rec': 7, 'rec_yards': 82, 'air...","{'targets': 7, 'rec': 5, 'rec_yards': 56, 'air...","{'targets': 8, 'rec': 6, 'rec_yards': 153, 'ai...","{'targets': 7, 'rec': 4, 'rec_yards': 24, 'air...","{'targets': 9, 'rec': 4, 'rec_yards': 101, 'ai..."
1,A.J. Green,WR,CIN,"{'targets': 9, 'rec': 5, 'rec_yards': 51, 'air...","{'targets': 5, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 9, 'rec': 4, 'rec_yards': 41, 'air...","{'targets': 3, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 1, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 7, 'rec': 6, 'rec_yards': 62, 'air...","{'targets': 3, 'rec': 2, 'rec_yards': 40, 'air...","{'targets': 7, 'rec': 4, 'rec_yards': 64, 'air...","{'targets': 6, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 13, 'rec': 3, 'rec_yards': 29, 'ai...","{'targets': 6, 'rec': 5, 'rec_yards': 36, 'air...","{'targets': 5, 'rec': 1, 'rec_yards': 3, 'air_...","{'targets': 1, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 11, 'rec': 8, 'rec_yards': 96, 'ai...","{'targets': 13, 'rec': 7, 'rec_yards': 82, 'ai...","{'targets': 5, 'rec': 2, 'rec_yards': 19, 'air...",NaN
2,Aaron Jones,RB,GB,"{'targets': 6, 'rec': 4, 'rec_yards': 10, 'air...","{'targets': 6, 'rec': 5, 'rec_yards': 49, 'air...","{'targets': 4, 'rec': 4, 'rec_yards': 30, 'air...","{'targets': 2, 'rec': 1, 'rec_yards': 0, 'air_...","{'targets': 3, 'rec': 3, 'rec_yards': 18, 'air...","{'targets': 3, 'rec': 2, 'rec_yards': 6, 'air_...","{'targets': 4, 'rec': 3, 'rec_yards': 13, 'air...","{'targets': 3, 'rec': 2, 'rec_yards': 14, 'air...","{'targets': 5, 'rec': 4, 'rec_yards': 43, 'air...","{'targets': 8, 'rec': 4, 'rec_yards': 68, 'air...","{'targets': 4, 'rec': 2, 'rec_yards': 17, 'air...","{'targets': 5, 'rec': 5, 'rec_yards': 40, 'air...",NaN,"{'targets': 5, 'rec': 3, 'rec_yards': 26, 'air...",NaN,NaN,"{'targets': 5, 'rec': 5, 'rec_yards': 21, 'air..."
3,Aaron Rodgers,QB,GB,"{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 1, 'rec': 1, 'rec_yards': -6, 'air...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...",NaN,"{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_...","{'targets': 0, 'rec': 0, 'rec_yards': 0, 'air_..."
4,Adam Humphries,WR,TEN,"{'targets': 7, 'rec': 6, 'rec_yards': 47, 'air...",NaN,NaN,NaN,"{'targets': 4, 'rec': 1, 'rec_yards': 9, 'air_...",NaN,NaN,NaN,NaN,"{'targets': 6, 'rec': 5, 'rec_yards': 48, 'air...","{'targets': 7, 'rec': 4, 'rec_yards': 41, 'air...",NaN,NaN,"{'targets': 6, 'rec': 6, 'rec_yards': 64, 'air...","{'targets': 3, 'rec': 1, 'rec_yards': 19, 'air...","{'targets': 2, 'rec': 0, 'rec_yards': 0, 'air_...",NaN
5,Adam Thielen,WR,MIN,"{'targets': 8, 'rec': 6, 'rec_yards': 110, 'ai...","{'targets': 7, 'rec': 4, 'rec_yards': 43, 'air...","{'targets': 11, 'rec': 8, 'rec_yards': 123, 'a...",NaN,"{'targets': 11, 'rec': 8,